In [ ]:
from google.colab import drive
import os
import cv2
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Mount Google Drive
drive.mount('/content/drive')

# Image size
size = 100

# Load training images - Pothole
potholeTrainImages = [os.path.join("/content/drive/MyDrive/Dataset/train/Pothole", img)
                      for img in os.listdir("/content/drive/MyDrive/Dataset/train/Pothole")
                      if img.endswith((".jpg", ".jpeg", ".png"))]
train1 = [cv2.resize(cv2.imread(img), (size, size)) for img in potholeTrainImages if cv2.imread(img) is not None]
temp1 = np.asarray(train1)

# Load training images - Plain
nonPotholeTrainImages = [os.path.join("/content/drive/MyDrive/Dataset/train/Plain", img)
                         for img in os.listdir("/content/drive/MyDrive/Dataset/train/Plain")
                         if img.endswith((".jpg", ".jpeg", ".png"))]
train2 = [cv2.resize(cv2.imread(img), (size, size)) for img in nonPotholeTrainImages if cv2.imread(img) is not None]
temp2 = np.asarray(train2)

# Load testing images - Pothole
potholeTestImages = [os.path.join("/content/drive/MyDrive/Dataset/test/Pothole", img)
                     for img in os.listdir("/content/drive/MyDrive/Dataset/test/Pothole")
                     if img.endswith((".jpg", ".jpeg", ".png"))]
test1 = [cv2.resize(cv2.imread(img), (size, size)) for img in potholeTestImages if cv2.imread(img) is not None]
temp3 = np.asarray(test1)

# Load testing images - Plain
nonPotholeTestImages = [os.path.join("/content/drive/MyDrive/Dataset/test/Plain", img)
                        for img in os.listdir("/content/drive/MyDrive/Dataset/test/Plain")
                        if img.endswith((".jpg", ".jpeg", ".png"))]
test2 = [cv2.resize(cv2.imread(img), (size, size)) for img in nonPotholeTestImages if cv2.imread(img) is not None]
temp4 = np.asarray(test2)

# Combine and label data
X_train = np.concatenate((temp1, temp2))
X_test = np.concatenate((temp3, temp4))
y_train = np.concatenate((np.ones(temp1.shape[0], dtype=int), np.zeros(temp2.shape[0], dtype=int)))
y_test = np.concatenate((np.ones(temp3.shape[0], dtype=int), np.zeros(temp4.shape[0], dtype=int)))

# Shuffle
X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

# One-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Define model
def kerasModelVGG16(size=100):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(size, size, 3))
    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(2, activation='softmax'))
    for layer in base_model.layers:
        layer.trainable = False
    return model

# Compile and train model
model = kerasModelVGG16(size)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_split=0.1)

# Evaluate model
metrics = model.evaluate(X_test, y_test)
for metric_i in range(len(model.metrics_names)):
    print(f"{model.metrics_names[metric_i]}: {metrics[metric_i]}")

# Save model
print("Saving model weights and configuration file")
model.save('sample_vgg16.h5')
model_json = model.to_json()
with open("sample_vgg16.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("sample_vgg16.weights.h5")
print("Saved model to disk")

# Predict and print F1 score
y_pred_probs = model.predict(X_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=["Plain", "Pothole"]))

f1 = f1_score(y_true_classes, y_pred_classes)
print("F1 Score (binary):", f1)

cm = confusion_matrix(y_true_classes, y_pred_classes)
print("\nConfusion Matrix:")
print(cm)


Mounted at /content/drive
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 80s 4s/step - accuracy: 0.5687 - loss: 4.5326 - val_accuracy: 0.9000 - val_loss: 0.4641
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 80s 4s/step - accuracy: 0.8516 - loss: 0.7538 - val_accuracy: 0.9143 - val_loss: 0.2724
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - accuracy: 0.9225 - loss: 0.3008 - val_accuracy: 0.9429 - val_loss: 0.2597
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - accuracy: 0.9491 - loss: 0.2681 - val_accuracy: 0.9571 - val_loss: 0.2724
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 80s 4s/step - accuracy: 0.9535 - loss: 0.1553 - val_accuracy: 0.9571 - val_loss: 0.2679
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 81s 4s/step - accuracy: 0.9672 - loss: 0.1125 - val_accuracy: 0.9571 - val_loss: 0.2970
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 85s 4s/step - accuracy: 0.9741 - loss: 0.0896 - val_accuracy: 0.9571 - val_loss: 0.2766
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 78s

loss: 1.1733657121658325
compile_metrics: 0.8125
Saving model weights and configuration file
Saved model to disk
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

Classification Report:
              precision    recall  f1-score   support

       Plain       0.86      0.75      0.80         8
     Pothole       0.78      0.88      0.82         8

    accuracy                           0.81        16
   macro avg       0.82      0.81      0.81        16
weighted avg       0.82      0.81      0.81        16

F1 Score (binary): 0.8235294117647058

Confusion Matrix:
[[6 2]
 [1 7]]
